In [7]:
import pandas as pd
import numpy as np
import scipy as sp

In [91]:
import seaborn as sns
from matplotlib import pyplot as plt

In [490]:
class UGMM(object):
    
    def __init__(self, X, K=1):
        self.X = X
        self.K = K
        self.N = self.X.shape[0]

    def _init(self):
        self.alpha=np.random.rand(self.K,1)
        self.m= np.random.rand(self.K,1)
        self.beta=np.random.rand(self.K,1)
        self.beta_prev=np.random.rand(self.K,1)
        self.nu=np.random.rand(self.K,1)*np.random.randint(1,10)
        self.W=np.random.rand(self.K,1)*np.random.randint(1,10)
        self.alpha=np.random.rand(self.K,1)
        self.r=np.zeros((self.N,self.K))
        self.x_nebla=np.zeros((self.K,1))
        self.Nk=np.zeros((self.K,1))
        self.S=np.zeros((self.K,1))
        self.pi_nebla=np.zeros((self.K,1))
        self.lamda_nebla=np.zeros((self.K,1))
        self.E_mu_lamda=np.zeros((self.N,self.K))
        
        

        print('Init mean')
        print(self.m)
        

    def E_Step(self):
        for n in range(self.N):
            for k in range(self.K):
                self.pi_nebla[k]=np.exp(sp.special.digamma(self.alpha[k])-sp.special.digamma(self.alpha.sum()))
                self.lamda_nebla[k]=np.exp(sp.special.digamma(self.nu[k]/2, out=None)+np.log(2)+np.log(self.W[k]))
                self.E_mu_lamda[n][k]=(self.beta[k]**(-1))+self.nu[k]*self.W[k]*(self.X[n]-self.m[k])**2
                self.r[n][k]=self.pi_nebla[k]*(self.lamda_nebla[k]**0.5)*(np.exp(-0.5*self.E_mu_lamda[n][k]))
        row_sums = self.r.sum(axis=1)
        self.r= self.r / row_sums[:, np.newaxis]
        
        
    def fit(self, max_iter=1000, tol=1e-6):
        self._init()
#         self.r_values = [self.r_elbo()]
        self.m_history = [self.m.copy()]
        self.E_Step()
#         self.s2_history = [self.s2]
        for iter_ in range(1, max_iter+1):
#             print('before')
#             print(self.m_history)
            self.M_Step()
#             print('after')
#             print(self.m_history)
            self.m_history.append(self.m.copy())
            if iter_ % 10 == 0:
                print(iter_, self.m_history[iter_])
                
            if np.linalg.norm(self.m_history[-1]-self.m_history[-2],2) <= tol:
                print('ELBO converged with ll %.3f at iteration %d'%(self.m_history[-1][0],iter_))
                break
            self.E_Step()
        if iter_ == max_iter:
            print('ELBO ended with ll %.3f'%(self.m_history[-1][0]))


    def M_Step(self):
        self._update_Nk()
        self._update_x_nebla()
        self._update_S()
        self._update_alpha()
        self._update_W()
        self._update_beta()
        self._update_nu()
        self._update_m()

    def _update_Nk(self):
        for k in range(self.K):
            self.Nk[k]=self.r[:,k].sum()
            
    def _update_x_nebla(self):
        for k in range(self.K):
            self.x_nebla[k]=0
            for n in range(self.N):
#                 print(self.Nk)
                self.x_nebla[k]+=self.r[n][k]*self.X[n]/self.Nk[k]
    
    def _update_S(self):
        for k in range(self.K):
            self.S[k]=0
            for n in range(self.N):
                self.S[k]+=(self.r[n][k]*(self.X[n]-self.x_nebla[k])**(2))/self.Nk[k]
                
    def _update_alpha(self):
        for k in range(self.K):
            self.alpha[k]=self.alpha[k]+self.Nk[k]
            
    def _update_beta(self):
        self.beta_prev=self.beta.copy()
        for k in range(self.K):
            self.beta[k]=self.beta[k]+self.Nk[k]
        
    def _update_m(self):
        for k in range(self.K):
            self.m[k]=(self.beta_prev[k]*self.m[k]+self.Nk[k]*self.x_nebla[k])/self.beta[k]
#         print('achha')
#         print(self.m)
    
    
    def _update_W(self):
        for k in range(self.K):
            self.W[k]=((1/self.W[k])+self.Nk[k]*self.S[k]+(self.beta[k]*self.Nk[k]/(self.beta[k]+self.Nk[k]))*(self.x_nebla[k]-self.m[k])**2)**(-1)
        
    
    def _update_nu(self):
        for k in range(self.K):
            self.nu[k]=self.nu[k]+self.Nk[k]
    
#     def _calculate_rnk
        

In [491]:
data=pd.read_csv('data2.txt',header=None)

In [492]:
X =np.array(data.iloc[0,0])
for i in range(1,1000):
    X= np.append(X, data.iloc[i,0])
    

In [493]:
X.max()

0.9942949885119834

In [ ]:
ugmm = UGMM(X, 3)
ugmm.fit()

Init mean
[[0.57424361]
 [0.64841092]
 [0.5363646 ]]
10 [[0.57424361]
 [0.47291647]
 [0.45148363]]
20 [[0.57424361]
 [0.47631089]
 [0.45141075]]
30 [[0.57424361]
 [0.47771849]
 [0.4513822 ]]
40 [[0.57424361]
 [0.47850582]
 [0.45136657]]
50 [[0.57424361]
 [0.47901432]
 [0.45135661]]
60 [[0.57424361]
 [0.47937221]
 [0.45134966]]
70 [[0.57424361]
 [0.47963899]
 [0.45134451]]
80 [[0.57424361]
 [0.47984625]
 [0.45134053]]
90 [[0.57424361]
 [0.48001235]
 [0.45133735]]


In [481]:
mean=ugmm.m

In [482]:
mean

array([[0.61638225],
       [0.45203307],
       [0.50524659]])